In [ ]:
entity_list = ['admiral','agri']
import pandas as pd


In [ ]:
entity_list

In [ ]:
! pip install keybert

In [ ]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT()

In [ ]:
import string

In [ ]:
def customtokenizer(text):
    text_list =  text.split()
    for punc in set(string.punctuation):
        if punc in text_list:
            text_list.remove(punc)
        text_list = [i.strip(punc) for i in text_list] 
    return text_list    

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer = CountVectorizer(ngram_range=(1,len(entity_list)), stop_words="english",tokenizer=customtokenizer)

In [ ]:
d1 = {}
new_df = pd.DatFrame([],columns=['entity','keyword'])
for element in entity_list:
    element = element.split()
    if len(element) > 2:
        n_minus_one_words = element[:-1]
    elif len(element) <= 2:
        n_minus_one_words = element
        remove_words = ['LLC','Corporation']
        n_minus_one_words = [elements for elements in n_minus_one_words if elements not in remove_words]
        print(element)
        print(n_minus_one_words)
        print(' '.join(n_minus_one_words))
        test = ' '.join(n_minus_one_words)
        first_word = n_minus_one_words[0].lower()
        doc = test
        extracted_kw = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,len(n_minus_one_words)),vectorizer=vectorizer)
        extracted_kw = [kw for kw in extracted_kw if kw[0].startswith(first_word)]
        d2[doc] = extracted_kw
        print("Extracted keywords:", extracted_kw)
        print("\n")
        for k,_ in extracted_kw:
            new_df_1.loc[len(new_df_1)] = [' '.join(element),k]
            

In [ ]:
frames = [new_df,new_df_1]
keyword_extracted = pd.concat(frames)

In [ ]:
import pandas as pd
df = pd.DataFrame([(key, value) for _, list_of_kw in d1.items() for key,value in list_of_kw], columns = ['keyword','probability']
df.to_csv('keywords_extracted_Cinven_1.csv', index= False)

In [ ]:
df = pd.read_csv("keyword_extracted_Cinven (1).csv")
df

In [ ]:
import requests 
import json

In [ ]:
url = 'https://gis.ey.net/api/search/search-entities'

no_of_hits_as_gup=[]

for keyword in df['keyword'].to_list():
    payload = {"SearchExternal": False,"StandardSearchParameters":{"SearchTerm": keyword,"UseBestFields":True,"IsFuzzy":False,"OnlyActive":True,"IncludeDebug":False}}
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        no_of_hits_as_gup.append(response.json()['TotalRecords']
    else:
        print(response.text)
                                 

In [ ]:
df['No_of_hits_as GUP'] = no_of_hits_as_gup
df_keywords_less__than_25  = df[df["No_of_hits_as_GUP"] < 25]

In [ ]:
df_keywords_less_than_25['No_of_words_in_keyword'] = df_keywords_less_than_25.apply(lambda x : len(x['keyword'].split()),axis=1)

In [ ]:
final_result_cinven = df_keywords_less_than_25.groupby('entity',sort=False).apply(lambda x : x.loc[(x['No_of_hits_as_GUP'] == X['No_of_hits_as_GUP'].max()) & (x['No_of_words_in_keyword'] == x['No_of_words_in_keyword'].min()),['keyword','No_of_hits as_gup']].iloc[0])

In [ ]:
Cinven_25 = final_resut_cinven.to_csv('keyword_25.csv',index=True)